<a href="https://colab.research.google.com/github/TJSun009/test_categorisation/blob/main/Test_Categorisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plan

In [ ]:
# Plan of Action

# Source-Test Mapping
# Get Graph Representation of Programs
# Create a Graph Neural Network Classfier to map src and test graph
# (Optional) Enhance graph tokens using GraphCodeBERT, CodeBERT or TREEBERT embeddings

# Test Generation
# Create a GraphTransformer using Graph Representations and Encoder-Decoder Architecture
# Prior Embeddings may be useful
# See GraphBERT - https://arxiv.org/abs/2001.05140
# Encoder - convert Graph nodes to a node embedding Representation based on surrounding nodes and edges

# Use Masked Node Modelling to Mask a Node in the AST and Generate it based on it's connected nodes and edges

## Mount Google Drive

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null

In [21]:
!sudo apt remove google-drive-ocamlfuse

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following packages will be REMOVED:
  google-drive-ocamlfuse
0 upgraded, 0 newly installed, 1 to remove and 27 not upgraded.
After this operation, 7,023 kB disk space will be freed.
(Reading database ... 124125 files and directories currently installed.)
Removing google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null

In [ ]:
!google-drive-ocamlfuse --help

In [19]:
service_account_path = "/content/southern-camera-367511-b575499bdaa5.json"
service_account_user = "colab-gdrive-auth@southern-camera-367511.iam.gserviceaccount.com"

!mkdir /content/drive
!google-drive-ocamlfuse -d -serviceaccountpath {service_account_path} -serviceaccountuser {service_account_user} "/content/drive"

mkdir: cannot create directory ‘/content/drive’: File exists
FUSE library version: 2.9.7
nullpath_ok: 0
nopath: 0
utime_omit_ok: 0
unique: 2, opcode: INIT (26), nodeid: 0, insize: 56, pid: 0
INIT: 7.32
flags=0x03fffffb
max_readahead=0x00020000
   INIT: 7.19
   flags=0x00000031
   max_readahead=0x00020000
   max_write=0x00020000
   max_background=0
   congestion_threshold=0
   unique: 2, success, outsize: 40
unique: 4, opcode: GETATTR (3), nodeid: 1, insize: 56, pid: 81
getattr /
   unique: 4, success, outsize: 120
unique: 6, opcode: ACCESS (34), nodeid: 1, insize: 48, pid: 81
   unique: 6, error: -38 (Function not implemented), outsize: 16
unique: 8, opcode: GETATTR (3), nodeid: 1, insize: 56, pid: 81
getattr /
   unique: 8, success, outsize: 120
^C


In [20]:
from google.colab import drive
drive.mount('/content/drive')

NotImplementedError: ignored

In [ ]:
!google-drive-ocamlfuse 

Starting application setup (label=default, base_dir=).
[0.000074] TID=0: Loading configuration from /root/.gdfuse/default/config...done
Opening log file: /root/.gdfuse/default/gdfuse.log


In [ ]:
!xdg-settings set default-web-browser google-chrome.desktop # to set default browser

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
!mkdir /content/drive
!google-drive-ocamlfuse /content/drive

xdg-settings: invalid application name
Try 'xdg-settings --help' for more information.
mkdir: cannot create directory ‘/content/drive’: File exists
^C


In [ ]:
!ls /content/drive/

In [ ]:
!rm -r /content/drive

In [ ]:
!rm -r ~/.gdfuse

In [ ]:
!fusermount -u /content/drive

## Local/Remote Environment

In [ ]:
import sys

DIR_PREFIX = "/content/drive/MyDrive/" if 'google.colab' in sys.modules else "/mnt/g/My Drive/"

if 'google.colab' not in sys.modules:
  SUDO = getpass("Enter your Sudo Password: ")

NotImplementedError: ignored

# neptune.ai Integration

In [ ]:
! pip install -Uqqq neptune-client
import neptune.new as neptune

neptune_api_token = getpass("Enter your Neptune API token: ")

project = "tjsun009/test-src-classifier"

run = neptune.init_run(
    api_token=neptune_api_token,
    project=project,
)


KeyboardInterrupt



# Source-Test Mapping

In [ ]:
! pip install -Uqqq scipy networkx

     |████████████████████████████████| 34.5 MB 163.3 MB/s 


## Import libraries

In [ ]:
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import pandas as pd
import networkx as nx
from glob import iglob
import importlib

### python-graphs dependency

In [ ]:
# New Graph Generator Approach
# https://arxiv.org/pdf/2208.07461v1.pdf

# install python-graphs on startup
! echo {SUDO} | sudo -S apt-get -qq -y install graphviz graphviz-dev
! pip install -Uqqq python-graphs gast

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 8.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libgtk2.0-common.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../0-libgtk2.0-common_2.24.32-1ubuntu1_all.deb ...
Unpacking libgtk2.0-common (2.24.32-1ubuntu1) ...
Selecting previously unselected package libgtk2.0-0:amd64.
Preparing to unpack .../1-libgtk2.0-0_2.24.32-1ubuntu1_amd64.deb ...
Unpacking libgtk2.0-0:amd64 (2.24.32-1ubuntu1) ...
Selecting previously unselected package libgail18:amd64.
Preparing to unpack .../2-libgail18_2.24.32-1ubuntu1_amd64.deb

#### Understanding python-graphs Representation

In [ ]:
# import gast as ast
# from python_graphs import program_graph

# # example file
# file_path = DIR_PREFIX + "Year 3/Dissertation/Projects/Datasets/data/minified/src/unittest_utils.py"

# with open(file_path, "r") as f:
#   # graph = program_graph.get_program_graph(f.read())
  
#   # read ast head
#   graph = program_graph.get_program_graph(ast.parse(f.read()))

In [ ]:
# # nodes are stored in a dictionary representing a key-value pair of the node id and node itself
# example_node_dict = graph.nodes

# # nodes are represented as strings by joining the node id and node ast_type if it has one 
# example_node_dict_item = list(example_node_dict.items())[0]

# # item 0 is the node id and item 1 the node representation
# example_node_dict_item

In [ ]:
# # as above the node is the value
# example_node = example_node_dict_item[1]

# # we can view the nodes properties as well
# # the ast_value is of particular interest as well for retrieving tokens
# # not all nodes will have a value though
# print(example_node.__dict__)
# print(example_node.__dict__["ast_node"].__dict__)

In [ ]:
# # show what ast_values look like
# nodes = list(graph.all_nodes())

# node_values = []
# for node in nodes:
#   if node.ast_value:
#     node_values.append(node.ast_value)
# # some ast_values are long strings so will need subtokens which can be combined
# # may require CodeBERT embeddings
# node_values

In [ ]:
# # check edge
# example_edge = graph.edges[0]
# example_edge

### CodeBERT dependency

In [ ]:
# imports for tokenising code values
! pip install -Uqqq transformers

     |████████████████████████████████| 5.8 MB 6.2 MB/s 
     |████████████████████████████████| 7.6 MB 87.1 MB/s 
     |████████████████████████████████| 182 kB 106.3 MB/s 


#### Investigating CodeBERT Tokeniser for code

In [ ]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
# model = AutoModel.from_pretrained("microsoft/codebert-base")

In [ ]:
# # join node values into single string
# code = (' ').join([str(val) for val in node_values])
# # code
# code_tokens = tokenizer.tokenize(code)
# # code_tokens
# tokens = [tokenizer.cls_token] + code_tokens + [tokenizer.sep_token]
# # tokens
# tokens_ids = tokenizer.convert_tokens_to_ids(tokens)

# # this is the final output from the model
# # it consists of 1 vector for each token across 768 distinct features 
# context_embeddings = model(torch.tensor(tokens_ids)[None,:])[0]
# # context_embeddings.shape
# len(code_tokens), context_embeddings.shape

# # in the graph embedding the node values could be represented as each value padded
# # by N others on either side, N would be calibrated for best results

## Prepare Dataset

produces graph_list (list of python_graphs)

In [ ]:
CODE_MINI_DIR = DIR_PREFIX + "Year 3/Dissertation/Projects/Datasets/data/minified"
CODE_LARGE_DIR = DIR_PREFIX + "Year 3/Dissertation/Projects/Datasets/data/large"
CODE_DIR = CODE_MINI_DIR

In [ ]:
# adapted from typilus for monitoring
class Monitoring:
    def __init__(self):
        self.count = 0  # type: int
        self.errors = []
        self.file = ""  # type: str
        self.empty_files = []

    def increment_count(self) -> None:
        self.count += 1

    def found_error(self, err, trace) -> None:
        self.errors.append([self.file, err, trace])

    def enter_file(self, filename: str) -> None:
        self.file = filename

In [ ]:
# adapted from typilus for generating graphs with python_graphs

from typing import Tuple, List, Optional, Set, Iterator
from python_graphs import program_graph

# TODO progress bar
# from tqdm.notebook import tqdm_notebook


# needed to use gast opposed to standard ast
import gast, ast
# importlib.reload(gast)
def explore_files(root_dir: str, monitoring: Monitoring) -> Iterator[Tuple]:
    """
    Walks through the root_dir and process each file.
    """
    for file_path in iglob(os.path.join(root_dir, '**', '*.py'), recursive=True):
        file_name = file_path.split('/').pop()
        if not os.path.isfile(file_path):
            continue
        # print(file_path)
        with open(file_path, encoding="utf-8", errors='ignore') as f:
            monitoring.increment_count()
            monitoring.enter_file(file_path)
            
            # difficulty parsing some files so had to be skipped
            try:
              graph = program_graph.get_program_graph(gast.parse(f.read()))
              
              # identify graph by file_name
              graph.filename = file_path[len(root_dir):]
              
              yield graph
            except:
              continue

In [ ]:
! pip install -Uqqq dill
import dill as pickle
# handle saving of graphs
def save_graphs(graph_list, dir = CODE_DIR):
  graph_dir = os.path.join(dir, "graphs", "")
  
  if not os.path.exists(graph_dir):
    os.makedirs(graph_dir)
  
  file = os.path.join(graph_dir, "graphs.pickle")

  pickle.dump(graph_list, open(file, "wb"), protocol = pickle.HIGHEST_PROTOCOL)

  return file

In [ ]:
graph_list = []
if not os.path.exists(os.path.join(CODE_DIR, "graphs", "graphs.pickle")):
  outputs = explore_files(CODE_DIR, monitoring)
  graph_list = [graph for graph in outputs]
  monitoring = Monitoring()
  save_graphs(graph_list)
  del(outputs)
  del(monitoring)
else:
  graph_list = pickle.load(open(os.path.join(CODE_DIR, "graphs", "graphs.pickle"), "rb"))

NameError: ignored

## Feed Data to Graph Network

### Creating CodeGraph Class

This class makes use of networkx a popular graph representation library

#### Imports

In [ ]:
# each edge should be weighted differently based on its type, edge should contain types
from python_graphs import program_graph_dataclasses

# for ast class list
import sys, inspect

# imports for tokenising code values
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

#### Node Feature Helpers

In [ ]:
# use enum value to set node type
# dict updates only keys contained therein
# automatic enum conversion
def node_types_to_ints(G):
  node_type_dict = nx.get_node_attributes(G, "node_type")
  int_dict = {k: {"node_type": v.value} for k, v in node_type_dict.items()}
  nx.set_node_attributes(G, int_dict)

# use enum class to convert value back
def ints_to_node_types(G):
  node_type_dict = nx.get_node_attributes(G, "node_type")
  node_type_dict = {k: {"node_type": program_graph_dataclasses.NodeType(v)} for k, v in node_type_dict.items()}
  nx.set_node_attributes(G, node_type_dict)

# ast type can be dealt with using string byte encoding
def ast_types_to_ints(G, ast_types):
  ast_type_dict = nx.get_node_attributes(G, "ast_type")
  int_dict = {k: {"ast_type": ast_types.index(v)} for k, v in ast_type_dict.items()}
  nx.set_node_attributes(G, int_dict)

def ints_to_ast_types(G, ast_types):
  ast_type_dict = nx.get_node_attributes(G, "ast_type")
  int_dict = {k: {"ast_type": ast_types[v]} for k, v in ast_type_dict.items()}
  nx.set_node_attributes(G, int_dict)

# ast_value embeddings done using CodeBERT embeddings
# N equates to context padding how many subsequent and following tokens are used in embedding
def ast_values_to_context_embeddings(G, vocab, N = 1):
  ast_value_dict = nx.get_node_attributes(G, "ast_value")

  embedding_dict = {}

  for k, v in ast_value_dict.items():

    v = str(v)

    idx = vocab.index(v)

    start, end = idx - N, idx + N

    if start > -1:
      if end - 1 > len(vocab):
        end = len(vocab) - 1
    else:
      start = 0
    
    code = ''.join(vocab[start : end])

    code_tokens = tokenizer.tokenize(code)

    tokens = [tokenizer.cls_token] + code_tokens + [tokenizer.sep_token]

    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)

    # this is the final output from the model
    # it consists of 1 vector for each token across 768 distinct features 
    context_embeddings = model(torch.tensor(tokens_ids)[None,:])[0]

    embedding_dict[k] = {"ast_value": context_embeddings}

  nx.set_node_attributes(G, embedding_dict)


#### Implementation

In [ ]:
# This code graph class represents a combination of all the graphs generated in the code corpus
# This code graph class represents a single graph generated in the code corpus
class CodeGraph:

  def __init__(self):
    self.G = nx.MultiDiGraph()
    self.vocab = set()
    self.identifier = ""
    self.is_pair = None
    self.types = {
        "edge" : program_graph_dataclasses.EdgeType._member_names_,
        "ast" : [cls for _, cls in inspect.getmembers(sys.modules["ast"], inspect.isclass)]
        }

  def read(self, graph):

    # set identifier to file_name of graph if graph is a module
    if (graph.root.ast_type == "Module"):
      self.identifier = graph.filename
    # TODO implement identifier for functions instead

    # add nodes to graph along with their attributes
    # dict comprehension deduplicates node id
    # we can exclude the ast_node as this info should be encoded in the graphs and edges
    # exclude instruction temporarily due to complexity
    nodes = graph.all_nodes()
    self.G.add_nodes_from([(node.id, {k: v for k, v in node.__dict__.items() if k not in ["id", "ast_node", "instruction"]}) for node in nodes])
    
    # append edges to the graph along with their attributes
    # dict comprehension deduplicates node ids for edge
    self.G.add_edges_from([(edge.id1, edge.id2, {k: v for k, v in edge.__dict__.items() if k.find("id") == -1 }) for edge in graph.edges])
    
    # add ast values to vocab
    self.vocab.update([str(token) for token in nx.get_node_attributes(self.G, "ast_value").values()])

  def node_feature_vector_graph(self):
    
    # some node features have been discarded as they are too complex to be used by pytorch
    # or replicate info stored elsewhere in the graph

    H = self.G

    node_type = H.nodes(data="node_type").values()[0]
    if not isinstance(node_type, int):
      node_types_to_ints(H)
      ast_types_to_ints(H, self.types.ast)
      ast_values_to_context_embeddings(H, self.vocab)
    
    return H

    

    """{'node_type': <NodeType.AST_NODE: 1>, 
      # ignoring instruction due to it being another complex graph
      'instruction': <python_graphs.instruction.Instruction object at 0x7ff853d756d0>, 
      'ast_type': 'Expr', 
      'ast_value': '', 
      'syntax': ''}"""
    
    # ast_value encoding
  
  def draw(self):
    if len(self.G.nodes) > 0:
      # create normalizer for colours
      norm = plt.Normalize()

      # use vocab and edge_types to generate colours for plot
      # edges are mapped to their position in types
      token_colors = [self.vocab.index(val) for val in list(nx.get_node_attributes(self.G, "ast_value").values())]
      edge_type_colors = [edge_type.value for edge_type in list(nx.get_edge_attributes(self.G, "type").values())]
      
      # normalize the colors between [0, 1]
      node_color, edge_color = norm(token_colors), norm(edge_type_colors)

      fig, ax = plt.subplots(1, 1, figsize=(10, 10));

      nx.draw_networkx(self.G, edge_color = edge_color, node_color = node_color, with_labels=True, ax = ax)
  
  def pytorch_graph(self):
    H = node_feature_vector_graph(self)
    
    pyg = from_networkx(H)
    
    if (self.is_pair):
      pyg.y = torch.tensor([int(self.is_pair)])
    
    return 

### Get a Graph from List

In [ ]:
# graph = graph_list[10]

# print(list(graph.nodes))
# print(len(graph.nodes))
# print(graph.root.ast_type)
# print(graph.__dict__)

# # came across a graph of size 4734
# # may be sign to split graphs by test instead
# # file_name, len(graph.nodes), len(cg.G.nodes)

# # split graphs by test
# # len(list(graph.get_ast_nodes_of_type(ast.FunctionDef.__name__)))

### Create a Code Graph List

In [ ]:
code_graph_list = []

for graph in graph_list:
  cg = CodeGraph()
  cg.read(graph)
  code_graph_list.append(cg)

# cleanup graph list
del(graph_list)

code_graph_list

In [ ]:
code_graph_list[0].__dict__

### Remove Transformers model and tokenizer from memory

In [ ]:
del(model)

In [ ]:
del(tokenizer)

### Pairing Graphs

In [ ]:
# dataset will consist of:
# graph - a graph containing  candidate src graph and test graph
# label - a 0 or 1 corresponding to whether the test and src are a valid pairing

# filter out combination results that contain two source files or two test files
def is_src_test_pair(pair):
  graph1, graph2 = pair

  # checks both are not the same type of file
  return graph1.identifier.find("test") != graph2.identifier.find("test")

# function for combining code_graphs
def combine_code_graphs(pair):
  code_graph1, code_graph2 = pair
  # check if the node vectorisation has already happened
  node_type_list = [
      list(code_graph1.G.nodes(data="node_type"))[0][1],
      list(code_graph2.G.nodes(data="node_type"))[0][1],
  ]
  
  if any([isinstance(node_type, int) for node_type in node_type_list]):
    raise Exception("Cannot combine code graphs that have already been vectorised")

  # uses number of nodes to verify combination graph worked correctly
  graph1, graph2 = code_graph1.G, code_graph2.G
  
  H = nx.disjoint_union(graph1, graph2)

  code_graph1.G = H
  code_graph1.vocab.update(list(code_graph2.vocab))

  # add a property to the code_graph checking whether or not there are a code, test pair
  # False for pair, True for non pair
  # dealing with file at present
  # mapping functions it will require looking at AST calls etc.
  code_graph1.is_pair = code_graph1.identifier.replace("_test.py", "") == code_graph2.identifier.replace("_test.py", "")

  # cleanup vars for concurrency
  del(graph1)
  del(graph2)
  del(code_graph2)
  del(node_type_list)
  del(H)
  return code_graph1

In [ ]:
def save_paired_code_graphs(graph_list, dir = CODE_DIR):
  graph_dir = os.path.join(dir, "graphs", "")
  
  if not os.path.exists(graph_dir):
    os.makedirs(graph_dir)
  
  file = os.path.join(graph_dir, "code_graphs.pickle")

  pickle.dump(graph_list, open(file, "wb"), protocol = pickle.HIGHEST_PROTOCOL)

  # cleanup
  del(graph_dir)
  del(file)

In [ ]:
! pip install bounded-pool-executor
# get all possible combinations of code_graphs
from itertools import combinations
from bounded_pool_executor import BoundedProcessPoolExecutor

# only regenerated paired graphs if not already pickled 
if not os.path.exists(os.path.join(CODE_DIR, "graphs", "code_graphs.pickle")):
  # update combined_code_graph_list to contain the combined code graphs with parallelisation
  # inline to reduce stored vars
  pairs = list(filter(is_src_test_pair, list(combinations(code_graph_list, 2))))
  del(code_graph_list)
  
  with BoundedProcessPoolExecutor(max_workers = 20) as executor:
    combined_code_graph_list = list(executor.map(combine_code_graphs, pairs, chunksize = 100))
  # cleanup code_graph_list
  del(pairs)
  del(executor)
  save_paired_code_graphs(combined_code_graph_list)
else:
  combined_code_graph_list = pickle.load(open(os.path.join(CODE_DIR, "graphs", "code_graphs.pickle"), "rb"))

### Stop Neptune

In [ ]:
# stop neptune
run.stop()

In [ ]:
# idea for doing tested source MLM
# provide the source graph as input and mask a node in the test graph randomly
# predict what the node is, including node_type, node_value if applicable 

In [ ]:
%debug

### PyTorch Conversion

In [ ]:
import torch
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 1.13.0+cu116


In [ ]:
!pip install -Uqqq torch-scatter torch-sparse torch-geometric -f https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html

In [ ]:
from torch_geometric.utils.convert import from_networkx



torch_graph = cg.pytorch_graph()


RuntimeError: ignored

## GCN Layer

In [ ]:
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv

# Graph Classifier Network Layer
# Learns embedding of a graph based on its nodes

class GCN(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels):
    super().__init__()
    self.conv1 = GCNConv(in_channels, hidden_channels)
    self.conv2 = GCNConv(hidden_channels, out_channels)

  def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
    # x: Node feature matrix of shape [num_nodes, in_channels]
    # edge_index: Graph connectivity matrix of shape [2, num_edges]
    x = self.conv1(x, edge_index).relu()
    x = self.conv2(x, edge_index)
    return x

## GCN Layer

In [ ]:
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv

# Graph Classifier Network Layer
# Learns embedding of a graph based on its nodes

class GCN(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels):
    super().__init__()
    self.conv1 = GCNConv(in_channels, hidden_channels)
    self.conv2 = GCNConv(hidden_channels, out_channels)

  def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
    # x: Node feature matrix of shape [num_nodes, in_channels]
    # edge_index: Graph connectivity matrix of shape [2, num_edges]
    x = self.conv1(x, edge_index).relu()
    x = self.conv2(x, edge_index)
    return x

## Create Train and Evaluation Experiment

In [ ]:
# Train and Arch variables
hidden_units = [32, 32]
learning_rate = 0.01
dropout_rate = 0.5
num_epochs = 300
batch_size = 256

In [ ]:
def display_learning_curves(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(history.history["loss"])
    ax1.plot(history.history["val_loss"])
    ax1.legend(["train", "test"], loc="upper right")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")

    ax2.plot(history.history["acc"])
    ax2.plot(history.history["val_acc"])
    ax2.legend(["train", "test"], loc="upper right")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    plt.show()